In [1]:
import io
import os
import re
from xml.dom import minidom
import json
import codecs

Navigate to recipe directory:

In [2]:
#f = io.open("igt_cat.json", mode="r", encoding="utf-8")
#cwd = os.getcwd()
# Print the current working directory
#print("Current working directory: {0}".format(cwd))
os.chdir('C:/Users/nlutt/Documents/Websites/ckCollections/recipes_xml/03 Mangold')
cwd = os.getcwd()
print("Current working directory: {0}".format(cwd))
# f.read()
os.listdir(cwd)
res = []
# Iterate directory
for path in os.listdir(cwd):
    # check if current path is a file
    if os.path.isfile(os.path.join(cwd, path)):
        res.append(path)
print(len(res))

Current working directory: C:\Users\nlutt\Documents\Websites\ckCollections\recipes_xml\03 Mangold
877


Find out file name parts:

In [3]:
s = "recipe-1021541207378794 - Ananas - Erdbeer - Konfitüre.xml"
id = re.match(r"recipe-(\d+)", s).group(1)
print (id)

1021541207378794


In [4]:
rn = re.match(r"recipe-\d+\s-\s(.+)\.xml",s).group(1)
rn = re.sub(r"\s-\s", '-' ,rn)
print (rn)

Ananas-Erdbeer-Konfitüre


In [5]:
fn_new = rn + ' - r' + id + '.xml'
print (fn_new)

Ananas-Erdbeer-Konfitüre - r1021541207378794.xml


### Find duplicate recipe names
Find names:

Renaming: done manually. Find here a still uncomplete procedure for automatic renaming (out-commented):

In [6]:
# Iterate directory
#for fn in os.listdir(cwd):
#    fn_new = re.match(r"(.*)\s-\sr", fn).group(1) + '.xml'
#    os.rename (fn, fn_new)
print (len(os.listdir(cwd)))

877


### Read and parse ingredient catalogue

In [2]:
with open('C:/Users/nlutt\Documents/Websites/Ingredients/igt-catalog.json', 'r', encoding='utf-8') as f:
  igtCat = json.load(f)
cat = igtCat.get('ingredients')
print (len(cat))
#for igt in cat:
#    print (igt.get('normName'))

720


### Access to ingredient names in recipes ("given" names)

In [4]:
no_match_ctr = 0
os.chdir('C:/Users/nlutt/Documents/Websites/ckCollections/recipes_xml/03 Mangold')
fn = "Schalottenpfannekuchen mit Ziegenkäse gefüllt.xml"
recipe = minidom.parse (fn)
ingredients = recipe.getElementsByTagName("fr:igdtName")
rc = []
def getIngredientNames(ingredients):
    for ingredient in ingredients:
        xx = ingredient.childNodes
        for x in xx:
            if x.nodeType == x.TEXT_NODE:
                rc.append(x.data)
getIngredientNames (ingredients)
print (rc)

['Schalotte(n)', 'Olivenöl', 'Mehl', 'Milch', 'Ei(er)', 'Butter, flüssige', 'Mangold', 'Lauch', 'Käse (schnittfester Ziegenkäse)', 'Maiskörner, vorgegarte (Dose)', 'Knoblauch', 'Petersilie, fein geschnittene', 'Salz und Pfeffer aus der Mühle', 'Cayennepfeffer', 'Muskat, frisch gerieben']


### Middle and inner loop
* inner loop: check, if single given ingredient name appears in ingredient catalogue
* middle loop: check for all given names in single recipe

Take care that the step above has been executed before you start the inner and middle loop!

In [9]:
#middle loop: recipe ingredient name fields
for igt_in_rcp in rc:
    given = igt_in_rcp.lower()
    oder = re.match(r"(.*)\soder\s",given)
    if oder != None:
        given = oder.group(1)
    matches = []
    myMatch = {}
    #inner loop: normName and synonymes
    for igt in cat:
        x = igt.get('normName')
        m = re.search (x, given)
        if m != None:
            start, end = m.span()
            if end-start > 0:
                #print (end-start, '  ', m.group() )
                matching = {}
                matching['name']   = igt.get('id')
                matching['pos']    = start
                matching['length'] = end - start
                matches.append(matching) 
        synos = igt.get('synonyms')
        if synos != None:
            for syno in synos:
                m = re.search (syno, given)
                if m != None:
                    start, end = m.span()
                    if end-start > 0:
                        #print (end-start, '  ', m.group() )
                        matching = {}
                        matching['name']   = igt.get('id')
                        matching['pos']    = start
                        matching['length'] = end - start
                        matches.append(matching)
    if len(matches) > 0:
        matches_sorted = sorted(matches, key=lambda x: (-x['length'], x['pos']))
        myMatch = matches_sorted[0]
        print ('+'.rjust(6) + '  ' + given.ljust(45) + myMatch['name'])   
    if myMatch == {}:
        no_match_ctr += 1
#        message = str(no_match_ctr).rjust(6) + '  ' + given.ljust(35) + recipe.getElementsByTagName("fr:recipeName")[0].firstChild.nodeValue
        message = str(no_match_ctr).rjust(6) + '  ' + given.ljust(60) + fn
        print (message)

     +  schalotte(n)                                 schalotten
     +  olivenöl                                     olivenöl
     +  mehl                                         weizenmehl
     +  milch                                        milch
     +  ei(er)                                       ei
     +  butter, flüssige                             butter
     +  mangold                                      mangold
     +  lauch                                        lauch
     +  käse (schnittfester ziegenkäse)              ziegenkäse
     +  maiskörner, vorgegarte (dose)                mais
     +  knoblauch                                    knoblauch
     +  petersilie, fein geschnittene                petersilie
     +  salz und pfeffer aus der mühle               
     +  cayennepfeffer                               cayennepfeffer
     +  muskat, frisch gerieben                      muskat


### Outer loop
* check for all recipes in collection

Take care that the step "Access to ingredient names in recipes" has been executed before you start the outer loop!

In [ ]:
def getIngredientNames(ingredients):
    for ingredient in ingredients:
        xx = ingredient.childNodes
        for x in xx:
            if x.nodeType == x.TEXT_NODE:
                rc.append(x.data)
                
i = 0
no_match_ctr = 0
os.chdir('C:/Users/nlutt/Documents/Websites/ckCollections/recipes_xml/03 Mangold')
cwd = os.getcwd()
for fn in os.listdir(cwd):
    #print (fn)
    recipe = minidom.parse (fn)
    ingredients = recipe.getElementsByTagName("fr:igdtName")
    rc = []
    getIngredientNames (ingredients)
    i += 1
    #middle loop: recipe ingredient name fields
    for igt_in_rcp in rc:
        given = igt_in_rcp.lower()
        oder = re.match(r"(.*)\soder\s",given)
        if oder != None:
            given = oder.group(1)
        matches = []
        myMatch = {}
        #inner loop: normName and synonymes
        for igt in cat:
            x = igt.get('normName')
            m = re.search (x, given)
            if m != None:
                start, end = m.span()
                if end-start > 0:
                    #print (end-start, '  ', m.group() )
                    matching = {}
                    matching['name']   = igt.get('id')
                    matching['pos']    = start
                    matching['length'] = end - start
                    matches.append(matching) 
            synos = igt.get('synonyms')
            if synos != None:
                for syno in synos:
                    m = re.search (syno, given)
                    if m != None:
                        start, end = m.span()
                        if end-start > 0:
                            #print (end-start, '  ', m.group() )
                            matching = {}
                            matching['name']   = igt.get('id')
                            matching['pos']    = start
                            matching['length'] = end - start
                            matches.append(matching)
        if len(matches) > 0:
            matches_sorted = sorted(matches, key=lambda x: (-x['length'], x['pos']))
            myMatch = matches_sorted[0]
    #        print ('+'.rjust(6) + '  ' + given.ljust(45) + myMatch['name'])   
        if myMatch == {}:
            no_match_ctr += 1
    #        message = str(no_match_ctr).rjust(6) + '  ' + given.ljust(35) + recipe.getElementsByTagName("fr:recipeName")[0].firstChild.nodeValue
            message = str(no_match_ctr).rjust(6) + '  ' + given.ljust(60) + fn
    #        print (message)

### Set ingredient ref attribute in single recipe
* middle and inner loop:

In [30]:
i = 0
no_match_ctr = 0
os.chdir('C:/Users/nlutt/Documents/Websites/ckCollections/recipes_xml/03 Mangold')
cwd = os.getcwd()
fn = "Schalottenpfannekuchen mit Ziegenkäse gefüllt.xml"
recipe = minidom.parse (fn)
ingredients = recipe.getElementsByTagName("fr:igdtName")

#middle loop: recipe ingredient name fields
for ingredient in ingredients:
    given = ingredient.firstChild.nodeValue.lower()
    #print (ingredient.getAttribute('ref'))
    oder = re.match(r"(.*)\soder\s",given)
    if oder != None:
        given = oder.group(1)
    matches = []
    myMatch = {}
    
    #inner loop: normName and synonymes
    for igt in cat:
        x = igt.get('normName')
        m = re.search (x, given)
        if m != None:
            start, end = m.span()
            if end-start > 0:
                #print (end-start, '  ', m.group() )
                matching = {}
                matching['name']   = igt.get('id')
                matching['pos']    = start
                matching['length'] = end - start
                matches.append(matching) 
        synos = igt.get('synonyms')
        if synos != None:
            for syno in synos:
                m = re.search (syno, given)
                if m != None:
                    start, end = m.span()
                    if end-start > 0:
                        #print (end-start, '  ', m.group() )
                        matching = {}
                        matching['name']   = igt.get('id')
                        matching['pos']    = start
                        matching['length'] = end - start
                        matches.append(matching)
    if len(matches) > 0:
        matches_sorted = sorted(matches, key=lambda x: (-x['length'], x['pos']))
        myMatch = matches_sorted[0]
        #print ('+'.rjust(6) + '  ' + given.ljust(45) + myMatch['name']) 
        ingredient.setAttribute ("ref", myMatch['name'])
        #print (ingredient.getAttribute('ref'))
    if myMatch == {}:
        no_match_ctr += 1
#        message = str(no_match_ctr).rjust(6) + '  ' + given.ljust(35) + recipe.getElementsByTagName("fr:recipeName")[0].firstChild.nodeValue
        message = str(no_match_ctr).rjust(6) + '  ' + given.ljust(60) + fn
#        print (message)

rn = recipe.getElementsByTagName("fr:recipeName")[0].firstChild
rn_new = rn.nodeValue[rn.nodeValue.index(" - ")+3:]
#print (rn_new) 
rn.replaceWholeText(rn_new)
#print(recipe.getElementsByTagName("fr:recipeName")[0].firstChild.nodeValue)
rcp_str = str(recipe.toxml())
#print(fn)
with codecs.open(fn, 'w', encoding = 'utf8') as file:
  file.write(rcp_str)

schalotten
     +  schalotte(n)                                 schalotten
schalotten
olivenöl
     +  olivenöl                                     olivenöl
olivenöl
weizenmehl
     +  mehl                                         weizenmehl
weizenmehl
milch
     +  milch                                        milch
milch
ei
     +  ei(er)                                       ei
ei
butter
     +  butter, flüssige                             butter
butter
mangold
     +  mangold                                      mangold
mangold
lauch
     +  lauch                                        lauch
lauch
ziegenkäse
     +  käse (schnittfester ziegenkäse)              ziegenkäse
ziegenkäse
mais
     +  maiskörner, vorgegarte (dose)                mais
mais
knoblauch
     +  knoblauch                                    knoblauch
knoblauch
petersilie
     +  petersilie, fein geschnittene                petersilie
petersilie

     +  salz und pfeffer aus der mühle               

cayennepfeffer

### Replace recipe name
New and old recipe name:

In [26]:
#rn = recipe.getElementsByTagName("fr:recipeName")[0].firstChild
#rn_new = rn.nodeValue[rn.nodeValue.index(" - ")+3:]
#print (rn_new) 
#rn.replaceWholeText(rn_new)
print(recipe.getElementsByTagName("fr:recipeName")[0].firstChild.nodeValue)

Schalottenpfannekuchen mit Ziegenkäse gefüllt


### In all recipes of collection, set references to igt catalogue 

In [3]:
no_match_ctr = 0
os.chdir('C:/Users/nlutt/Documents/Websites/ckCollections/recipes_xml/03 Mangold')
cwd = os.getcwd()

#outer loop: collection
for fn in os.listdir(cwd):
    if re.match(r"^\d\d", fn) == None:
        recipe = minidom.parse (fn)
        ingredients = recipe.getElementsByTagName("fr:igdtName")

        #middle loop: recipe 
        for ingredient in ingredients:
            given = ingredient.firstChild.nodeValue.lower()
            #print (ingredient.getAttribute('ref'))
            oder = re.match(r"(.*)\soder\s",given)
            if oder != None:
                given = oder.group(1)
            matches = []
            myMatch = {}

            #inner loop: ingredient names
            for igt in cat:
                x = igt.get('normName')
                m = re.search (x, given)
                if m != None:
                    start, end = m.span()
                    if end-start > 0:
                        #print (end-start, '  ', m.group() )
                        matching = {}
                        matching['name']   = igt.get('id')
                        matching['pos']    = start
                        matching['length'] = end - start
                        matches.append(matching) 
                synos = igt.get('synonyms')
                if synos != None:
                    for syno in synos:
                        m = re.search (syno, given)
                        if m != None:
                            start, end = m.span()
                            if end-start > 0:
                                #print (end-start, '  ', m.group() )
                                matching = {}
                                matching['name']   = igt.get('id')
                                matching['pos']    = start
                                matching['length'] = end - start
                                matches.append(matching)
            if len(matches) > 0:
                matches_sorted = sorted(matches, key=lambda x: (-x['length'], x['pos']))
                myMatch = matches_sorted[0]
                #print ('+'.rjust(6) + '  ' + given.ljust(45) + myMatch['name']) 
                ingredient.setAttribute ("ref", myMatch['name'])
                #print (ingredient.getAttribute('ref'))
            if myMatch == {}:
                no_match_ctr += 1
        #        message = str(no_match_ctr).rjust(6) + '  ' + given.ljust(35) + recipe.getElementsByTagName("fr:recipeName")[0].firstChild.nodeValue
                message = str(no_match_ctr).rjust(6) + '  ' + given.ljust(60) + fn
        #        print (message)

        #recipe name without introducing 'recipe13793749274 - '
        #rn = recipe.getElementsByTagName("fr:recipeName")[0].firstChild
        #rn_new = rn.nodeValue[rn.nodeValue.index(" - ")+3:]
        #print (rn_new) 
        #rn.replaceWholeText(rn_new)
        #print(recipe.getElementsByTagName("fr:recipeName")[0].firstChild.nodeValue)
        rcp_str = str(recipe.toxml())
        #print(fn)
        with codecs.open(fn, 'w', encoding = 'utf8') as file:
          file.write(rcp_str)

### In all recipes of collection, set recipe name to file name (without extension)

In [13]:
os.chdir('C:/Users/nlutt/Documents/Websites/ckCollections/recipes_xml/03 Mangold')
cwd = os.getcwd()

#outer loop: collection
for fn in os.listdir(cwd):
    if re.match(r"^\d\d", fn) == None:
        recipe = minidom.parse (fn)
        #recipe name without introducing 'recipe13793749274 - '
        rn = recipe.getElementsByTagName("fr:recipeName")[0].firstChild
        #rn_new = rn.nodeValue[rn.nodeValue.index(" - ")+3:]
        rn_new = re.match(r"^((.+-*)+)\.xml", fn).group(1)
        #print (rn_new) 
        rn.replaceWholeText(rn_new)
        print(recipe.getElementsByTagName("fr:recipeName")[0].firstChild.nodeValue)
        rcp_str = str(recipe.toxml())
        #print(fn)
        with codecs.open(fn, 'w', encoding = 'utf8') as file:
          file.write(rcp_str)

Adas bil Hamod
Albertos Mangold-Ravioli mit Chickenwings und Shiitake-Pilze
Albertos Mangold
Alex vegane Mangold-Kokosmilch-Suppe
Ananas-Erdbeer-Konfitüre
Anjas Käse-Mangold-Rinderrouladen
Apfel-Mangold-Tarte
Arabische Sommersuppe
Arap köftesi
Artischocken-Mangold-Dip
Asiatische Mangold-Nudeln mit Sesam
Asiatisches Mangold-Fisch Gratin
Asiatisches Schaschlik mit Mangold-Chutney
Aufgespießte Steakrollen
Auflauf mit Kartoffeln, Zucchini und Mangold
Augsburger Grießnockerlsuppe mit Gemüse
Avocado-Grapefruit Salat mit Paprikamandeln
Ayurvedische Kartoffel-Mangold-Zucchini-Pfanne
B&B Smoothie
Babymangoldsalat
Bambelsupp
Bandnudeln mit Mangold-Ingwer-Soße
Bandnudeln mit Mangold-Sahne-Sauce
Bauchlappensteak-Räder
Beefmakers Maultaschen
Bergkäse-Kugeln mit Mangold
Bitter Peach Symphonie
Blattgemüse mit Garnelen
Blattsalat mit Fisch und würzigem Gemüse
Blitva
Blätterteig-Quiche mit Brokkoli und Camembert
Blätterteig-Strudel mit Spargel und Putenfarce an Kräuter-Joghurt-Soße
Blätterteigtaschen m